<a href="https://colab.research.google.com/github/ldaniel-hm/eml_tabular/blob/main/EjemploGeneracionVideos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalación de algunos paquetes.
!apt-get update

# Para visualizar en colab https://medium.com/@coldstart_coder/visually-rendering-python-gymnasium-in-jupyter-notebooks-4413e4087a0f
!apt-get install -y python-opengl ffmpeg
!apt-get install --upgrade cmake

# Para usar gymnasium[box2d]
!apt install swig xvfb
!pip install -U  pyvirtualdisplay


In [ ]:
!pip install gymnasium gymnasium[box2d]

In [3]:
## Para visualizar en colab https://medium.com/@coldstart_coder/visually-rendering-python-gymnasium-in-jupyter-notebooks-4413e4087a0f
## https://medium.com/@coldstart_coder/dqn-algorithm-training-an-ai-to-land-on-the-moon-1a1307748ed9
# create a virtual display that will hold our video

# Google Colab se ejecuta en un entorno de servidor sin una interfaz gráfica de
# usuario (GUI). Para mostrar contenido visual como videos, se necesita una
# pantalla virtual para simular una pantalla.
# Esto permite que bibliotecas como gymnasium rendericen gráficos
# y los capturen para incrustarlos en el notebook.
#
# Este código crea una pantalla virtual oculta donde se puede renderizar contenido gráfico.

# pyvirtualdisplay es una biblioteca se usa para crear pantallas virtuales,
# que son esenciales para renderizar contenido gráfico en entornos sin interfaz gráfica.
from pyvirtualdisplay import Display

# visible=0 indica que la pantalla estará oculta (no visible para el usuario).
# Esto se debe a que Colab se ejecuta en un entorno de servidor sin una interfaz gráfica.
# size=(1400, 900) establece la resolución de la pantalla virtual a 1400x900 píxeles.
display = Display(visible=0, size=(1400, 900))

# inicia la pantalla virtual, preparándola para su uso.
display.start()


# Las siguientes líneas importan las bibliotecas necesarias para trabajar
# con entrada/salida, codificación base64 (usada para incrustar datos de video)
# y mostrar contenido HTML dentro del notebook de Colab.
import io
import base64
from IPython import display
from IPython.display import HTML

# Esta función mostrará un vídelo en la pantalla virtual, que será embembida en el notbook
def embed_video(video_file):
    # open and read the raw data from the video
    video_data = io.open(video_file, 'r+b').read()
    # now we have to encode the data into base64 to work
    # with the virtual display
    encoded_data = base64.b64encode(video_data)
    # now we use the display.display function to take some html
    # and the encoded data and embed the html into the notebook!
    display.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                </video>'''.format(encoded_data.decode('ascii'))))

In [7]:
import os # Para manipular archivos, directorios, variables de entorno y ejecutar comandos del SO

# 1. Define la carpeta donde se guardarán los vídeos
video_folder = "videos"  # Nombre de la carpeta (puedes cambiarlo)

# 2. Crea la carpeta si no existe
os.makedirs(video_folder, exist_ok=True) # exist_ok=True evita error si ya existe


In [9]:
import re # Para trabajar con expresiones regulares

def get_latest_episode_video_file(directory):
    # Expresión regular que coincide con el formato de los ficheros de video
    pattern = re.compile(r"rl-video-episode-(\d+)\.mp4")
    latest_file = None
    highest_episode = -1

    # Busca en el directorio
    for filename in os.listdir(directory):
        match = pattern.match(filename)
        if match:
            episode_number = int(match.group(1))  # Extrae el número de episodio
            # Comprobamos, para conseguir el número de episodio más alto.
            if episode_number > highest_episode:
                highest_episode = episode_number
                latest_file = os.path.join(directory, filename)  # Almacena el path completo

    return latest_file

In [10]:
# https://medium.com/@coldstart_coder/visually-rendering-python-gymnasium-in-jupyter-notebooks-4413e4087a0f
import gymnasium as gym
from gymnasium.wrappers import RecordEpisodeStatistics, RecordVideo

# 3. Crea el entorno LunarLander-v3
render = "rgb_array"
env = gym.make("FrozenLake-v1", map_name="8x8", is_slippery=False, render_mode=render) # render_mode "rgb_array" es necesario para RecordVideo


# 4. Aplica el wrapper RecordVideo para grabar vídeos
trigger = lambda episode_id: episode_id % 4 == 0 # small function that tells the recording env when to record, in our case always
env = gym.wrappers.RecordVideo(
    env,
    video_folder=video_folder,
    episode_trigger=trigger #, #(episode_id + 0) % 5 == 0, # Graba todos los episodios
    #name_prefix="lander_episode" # Prefijo para los nombres de los archivos de vídeo
    # name_prefix=lambda episode_id: f"lander_episodio_{(episode_id + 1):03d}_"  # Función lambda en name_prefix
)

#env = RecordVideo(env, './video')

num_episodes = 9 # Número de episodios a grabar
#observation, info = env.reset(seed=42)
for episode in range(num_episodes):
    observation, info = env.reset()
    episode_over = False
    while not episode_over:
        # Renderiza el frame actual, esto lo guardará en el archivo de video.
        env.render()

        # Selecciona una acción aleatoria del espacio de acciones disponible.
        # Esto simula la política del agente, que en este caso es aleatoria.
        action = env.action_space.sample()

        # Ejecuta un paso en el entorno con la acción seleccionada.
        # Obtiene la siguiente observación, la recompensa,
        # y si el episodio ha terminado (terminated) o ha sido truncado (truncated).
        observation, reward, terminated, truncated, info = env.step(action)

        # Verifica si el episodio ha terminado, ya sea por terminación o truncamiento.
        episode_over = terminated or truncated


env.close() # Importante cerrar el entorno, ¡esto finaliza la grabación de vídeo!
print(f"Grabación de episodios completada. Vídeos guardados en la carpeta '{video_folder}'")






/usr/local/lib/python3.11/dist-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /content/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Grabación de episodios completada. Vídeos guardados en la carpeta 'videos'


In [11]:
latest_file = get_latest_episode_video_file(video_folder)
print(f"Último vídeo: {latest_file}") # outputs: ./video/rl-video-episode-4.mp4
embed_video(latest_file)

Último vídeo: videos/rl-video-episode-8.mp4
